make sure that each product id of a call in a visit is indeed present in products collection

In [ ]:
# make sure that each product id of a call in a visit is indeed present in products collection
products_ids = [product['_id'] for product in products]

product_invalid_id = []
product_valid_id = []
product_no_id = 0
for i, visit in enumerate(visits):
    if i % 1000 == 0:
        print(i, len(product_invalid_id), len(set(product_invalid_id)), len(product_valid_id), len(set(product_valid_id)))
    for call in visit['calls']:
        if 'id' not in call['product']:
            product_no_id += 1
        elif call['product']['id'] not in products_ids:
            product_invalid_id.append(call['product']['id'])
        else:
            product_valid_id.append(call['product']['id'])

In [ ]:
# Define the pipeline
pipeline = [
    {
        '$unwind': '$calls'
    }, 
    {
        '$group': {
            '_id': {
                'user_id': '$user.id', 
                'product_id': '$calls.product.id'
            }, 
            'average_duration': {
                '$avg': '$calls.duration'
            }
        }
    }, 
    {
        '$group': {
            '_id': '$_id.user_id', 
            'products': {
                '$push': {
                    'product_id': '$_id.product_id', 
                    'average_duration': '$average_duration'
                }
            }
        }
    }
]